# RAGAS 기초 예제 - Context Recall
## 작성자 : AISchool ( http://aischool.ai/%ec%98%a8%eb%9d%bc%ec%9d%b8-%ea%b0%95%ec%9d%98-%ec%b9%b4%ed%85%8c%ea%b3%a0%eb%a6%ac/ )
## Reference : https://github.com/explodinggradients/ragas

In [ ]:
!pip install ragas langsmith nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.5/91.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.5/366.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import nest_asyncio

In [ ]:
# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# API Key 설정

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "여러분의_OPENAI_API_KEY"

In [ ]:
from uuid import uuid4

unique_id = uuid4().hex[0:8]

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"RAGAS Example - Context Recall - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "여러분의_LANGSMITH_API_KEY"

In [ ]:
unique_id

'fa76dd0f'

# 4. Context Recall 성능 측정
## Reference : https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html

In [ ]:
from ragas.metrics import context_precision, answer_relevancy, faithfulness, context_recall
from ragas import evaluate

In [ ]:
from datasets import Dataset

data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts' : [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'],
    ['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}
dataset = Dataset.from_dict(data_samples)

In [ ]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 2
})

In [ ]:
result = evaluate(
    dataset,
    metrics=[context_recall],
)

result

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

{'context_recall': 0.5000}

In [ ]:
result

{'context_recall': 0.5000}

# Context Recall 측정 프롬프트 살펴보기

In [ ]:
# question: When was the first super bowl?
# answer: The first superbowl was held on Jan 15, 1967
# contexts:
#   - The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,
# ground_truth: The first superbowl was held on January 15, 1967

In [ ]:
# context_recall: 1

In [ ]:
# Given a context, and an answer, analyze each sentence in the answer and classify if the sentence can be attributed to the given context or not. Use only "Yes" (1) or "No" (0) as a binary classification. Output json with reason.

# The output should be a well-formatted JSON instance that conforms to the JSON schema below.

# As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
# the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

# Here is the output JSON schema:
# ```
# {"type": "array", "items": {"$ref": "#/definitions/ContextRecallClassificationAnswer"}, "definitions": {"ContextRecallClassificationAnswer": {"title": "ContextRecallClassificationAnswer", "type": "object", "properties": {"statement": {"title": "Statement", "type": "string"}, "attributed": {"title": "Attributed", "type": "integer"}, "reason": {"title": "Reason", "type": "string"}}, "required": ["statement", "attributed", "reason"]}}}
# ```

# Do not return any preamble or explanations, return only a pure JSON string surrounded by triple backticks (```).

# Examples:

# question: "What can you tell me about albert Albert Einstein?"
# context: "Albert Einstein (14 March 1879 - 18 April 1955) was a German-born theoretical physicist, widely held to be one of the greatest and most influential scientists of all time. Best known for developing the theory of relativity, he also made important contributions to quantum mechanics, and was thus a central figure in the revolutionary reshaping of the scientific understanding of nature that modern physics accomplished in the first decades of the twentieth century. His mass-energy equivalence formula E = mc2, which arises from relativity theory, has been called 'the world's most famous equation'. He received the 1921 Nobel Prize in Physics 'for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect', a pivotal step in the development of quantum theory. His work is also known for its influence on the philosophy of science. In a 1999 poll of 130 leading physicists worldwide by the British journal Physics World, Einstein was ranked the greatest physicist of all time. His intellectual achievements and originality have made Einstein synonymous with genius."
# answer: "Albert Einstein born in 14 March 1879 was  German-born theoretical physicist, widely held to be one of the greatest and most influential scientists of all time. He received the 1921 Nobel Prize in Physics for his services to theoretical physics. He published 4 papers in 1905.  Einstein moved to Switzerland in 1895"
# classification: ```[{"statement": "Albert Einstein, born on 14 March 1879, was a German-born theoretical physicist, widely held to be one of the greatest and most influential scientists of all time.", "attributed": 1, "reason": "The date of birth of Einstein is mentioned clearly in the context."}, {"statement": "He received the 1921 Nobel Prize in Physics for his services to theoretical physics.", "attributed": 1, "reason": "The exact sentence is present in the given context."}, {"statement": "He published 4 papers in 1905.", "attributed": 0, "reason": "There is no mention about papers he wrote in the given context."}, {"statement": "Einstein moved to Switzerland in 1895.", "attributed": 0, "reason": "There is no supporting evidence for this in the given context."}]```

# question: "who won 2020 icc world cup?"
# context: "The 2022 ICC Men's T20 World Cup, held from October 16 to November 13, 2022, in Australia, was the eighth edition of the tournament. Originally scheduled for 2020, it was postponed due to the COVID-19 pandemic. England emerged victorious, defeating Pakistan by five wickets in the final to clinch their second ICC Men's T20 World Cup title."
# answer: "England"
# classification: ```[{"statement": "England won the 2022 ICC Men's T20 World Cup.", "attributed": 1, "reason": "From context it is clear that England defeated Pakistan to win the World Cup."}]```

# question: "What is the primary fuel for the Sun?"
# context: "NULL"
# answer: "Hydrogen"
# classification: ```[{"statement": "The Sun's primary fuel is hydrogen.", "attributed": 0, "reason": "The context contains no information"}]```

# Your actual task:

# question: When was the first super bowl?
# context: The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,
# answer: The first superbowl was held on January 15, 1967
# classification:

In [ ]:
# 주어진 맥락과 답변을 고려하여, 답변의 각 문장을 분석하고 해당 문장이 주어진 맥락에 기인할 수 있는지 여부를 분류합니다.
# "예" (1) 또는 "아니요" (0)로만 이진 분류를 사용하세요. 이유와 함께 JSON으로 출력하세요.

# 출력은 아래의 JSON 스키마에 맞는 잘 형식화된 JSON 인스턴스여야 합니다.

# 예를 들어, 스키마 {"properties":
# {"foo": {"title": "Foo", "description": "문자열 목록",# "type": "array", "items": {"type": "string"}}},
# "required": ["foo"]}에 대해 객체 {"foo": ["bar", "baz"]}는 스키마에 잘 맞는 인스턴스입니다. 객체 {"properties": {"foo": ["bar", "baz"]}}는 잘 맞지 않습니다.

# 다음은 출력 JSON 스키마입니다:
# ```
# {"type": "array", "items": {"$ref": "#/definitions/ContextRecallClassificationAnswer"},
# "definitions": {"ContextRecallClassificationAnswer": {"title": "ContextRecallClassificationAnswer",
# "type": "object", "properties": {"statement": {"title": "Statement", "type": "string"},
# "attributed": {"title": "Attributed", "type": "integer"}, "reason": {"title": "Reason", "type": "string"}}, "required": ["statement", "attributed", "reason"]}}}
# ```

# 어떤 서문이나 설명도 반환하지 말고, 삼중 백틱(```)으로 둘러싸인 순수한 JSON 문자열만 반환하세요.

# 예시:

# 질문: "알버트 아인슈타인에 대해 무엇을 알려줄 수 있나요?"
# 맥락: "알버트 아인슈타인 (1879년 3월 14일 - 1955년 4월 18일)은 독일 태생의 이론물리학자로,
# 역사상 가장 위대하고 영향력 있는 과학자 중 한 명으로 널리 알려져 있습니다. 상대성 이론을 개발한 것으로 가장 잘 알려져 있으며,
# 양자 역학에도 중요한 기여를 하여 20세기 첫 수십 년 동안 과학적 자연 이해를 혁신적으로 재편성하는 중심 인물이 되었습니다.
# 그의 질량-에너지 등가 공식 E = mc²는 상대성 이론에서 나왔으며 '세계에서 가장 유명한 방정식'이라고 불립니다.
# 그는 1921년 '이론물리학에 대한 공로와 특히 광전 효과 법칙의 발견'으로 노벨 물리학상을 받았으며, 이는 양자 이론의 발전에 중요한 단계였습니다. 그의 연구는 과학 철학에도 영향을 미친 것으로 알려져 있습니다.
# 1999년 영국 저널 Physics World가 전 세계 130명의 주요 물리학자를 대상으로 한 설문 조사에서 아인슈타인은 역대 가장 위대한 물리학자로 선정되었습니다. 그의 지적 업적과 독창성은 아인슈타인을 천재의 대명사로 만들었습니다."
# 답변: "알버트 아인슈타인은 1879년 3월 14일에 태어난 독일 태생의 이론물리학자로, 역사상 가장 위대하고 영향력 있는 과학자 중 한 명으로 널리 알려져 있습니다.
# 그는 이론물리학에 대한 공로로 1921년 노벨 물리학상을 받았습니다. 그는 1905년에 4편의 논문을 발표했습니다. 아인슈타인은 1895년에 스위스로 이주했습니다."
# 분류: [{"statement": "알버트 아인슈타인은 1879년 3월 14일에 태어난 독일 태생의 이론물리학자로, 역사상 가장 위대하고 영향력 있는 과학자 중 한 명으로 널리 알려져 있습니다.",
# "attributed": 1, "reason": "아인슈타인의 생년월일이 맥락에 명확히 언급되어 있습니다."},
# {"statement": "그는 이론물리학에 대한 공로로 1921년 노벨 물리학상을 받았습니다.", "attributed": 1, "reason": "정확한 문장이 주어진 맥락에 있습니다."},
# {"statement": "그는 1905년에 4편의 논문을 발표했습니다.", "attributed": 0, "reason": "주어진 맥락에는 그가 쓴 논문에 대한 언급이 없습니다."},
# {"statement": "아인슈타인은 1895년에 스위스로 이주했습니다.", "attributed": 0, "reason": "주어진 맥락에는 이에 대한 지원 증거가 없습니다."}]

# 질문: "2020년 ICC 월드컵 우승자는 누구인가요?"
# 맥락: "2022년 ICC 남자 T20 월드컵은 2022년 10월 16일부터 11월 13일까지 호주에서 열렸으며,
# 이 대회의 여덟 번째 판이었습니다. 원래 2020년으로 예정되었으나 COVID-19 팬데믹으로 인해 연기되었습니다.
# 잉글랜드는 결승전에서 파키스탄을 5개의 위켓 차이로 이기고 두 번째 ICC 남자 T20 월드컵 타이틀을 차지했습니다."
# 답변: "잉글랜드"
# 분류: [{"statement": "잉글랜드는 2022년 ICC 남자 T20 월드컵에서 우승했습니다.", "attributed": 1, "reason": "맥락에서 잉글랜드가 파키스탄을 이기고 월드컵에서 우승한 것이 명확합니다."}]

# 질문: "태양의 주요 연료는 무엇인가요?"
# 맥락: "NULL"
# 답변: "수소"
# 분류: [{"statement": "태양의 주요 연료는 수소입니다.", "attributed": 0, "reason": "맥락에 정보가 없습니다."}]

# Your actual task:

# question: When was the first super bowl?
# context: The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,
# answer: The first superbowl was held on January 15, 1967
# classification:

In [ ]:
# ```[{"statement": "The first Super Bowl was held on January 15, 1967.",
# "attributed": 1, "reason": "The exact date of the first Super Bowl is mentioned in the context."}]```

In [ ]:
# [{"statement": "첫 번째 슈퍼볼은 1967년 1월 15일에 열렸습니다.",
# "attributed": 1, "reason": "첫 번째 슈퍼볼의 정확한 날짜가 맥락에 언급되어 있습니다."}]